In [ ]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader,DirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

In [ ]:
import glob

# Spécifiez le chemin du répertoire contenant les fichiers PDF
directory_path = "Cleaned"

try:
    # Utilisez glob.glob pour obtenir une liste de tous les fichiers PDF dans le répertoire et ses sous-répertoires
    all_files = glob.glob(directory_path + "/*", recursive=True)

    # Utilisez votre loader pour charger tous les documents PDF récupérés
    loader = DirectoryLoader(directory_path,
                             loader_cls=TextLoader,
                             show_progress=True,
                             use_multithreading=True)

    documents = loader.load()

except Exception as e:
    print("Une erreur s'est produite lors de la lecture d'un fichier PDF :", e)
    print("Le fichier qui a provoqué l'erreur sera ignoré.")

In [ ]:
documents

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=10)
all_splits = text_splitter.split_documents(documents)

In [ ]:
all_splits

In [ ]:
#modelPath = 'OrdalieTech/Solon-embeddings-large-0.1'
modelPath = 'Lajavaness/bilingual-embedding-large'
model_kwargs ={
    'device':'cuda',
    'trust_remote_code':True
    }

encode_kwargs = {'normalize_embeddings':True}
# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

In [ ]:
db = FAISS.from_documents(all_splits, embeddings)


In [ ]:
# Définir le chemin du dossier où vous souhaitez sauvegarder l'index
folder_path = "Chatbot_RHV2"

# Nom facultatif pour l'index (par défaut, il sera nommé 'index')
index_name = "fiche_Chatbot_RH_docs"

# Enregistrer l'index sur le disque
db.save_local(folder_path, index_name)